In [ ]:
import skimage as ski
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from PIL import Image
import os
from pathlib import Path
import tifffile
import scipy as sp
import re
import scipy as sp
import statsmodels

import cv2, nd2, os, glob, re, sys
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import skimage as ski
from tqdm import tqdm
from importlib import reload
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from pathlib import Path
import seaborn as sns
import scipy as sp

# custom
from spheroid_invasiveness.spheroid import Spheroid

In [ ]:
# gloabls for figure making
DPI = 300

# directories
out_dir = "../output"
data_dir = "../data"
image_dir = os.path.join(data_dir, "raw/images/a3_images")
table_dir = os.path.join(data_dir, "raw/tables/")
t0_im_dir = os.path.join(image_dir, "A3_x10_0h")
t24_im_dir = os.path.join(image_dir, "A3_x10_24h")
t48_im_dir = os.path.join(image_dir, "A3_x10_48h")
t72_im_dir = os.path.join(image_dir, "A3_x10_72h")
t168_im_dir = os.path.join(image_dir, "A3_x10_168h")

data_dir = "../data/processed/images/invasiveness-images/"
out_dir = "../output/figures"

## Figure 1E

In [ ]:
# name which wells rotated or stayed stable
rotated_well_names = ['A1', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C2']
stable_well_names = ['A2', 'C1', 'C3', 'C4']

# map of the tumor:fibroblast ratios by well name
well_tumfib_ratio = {
    'A1': 10,
    'A2': 0.1,
    'A3': 1,
    'A4': 1,
    'B1': 5,
    'B2': 0.2,
    'B3': 1,
    'B4': np.inf,
    'C1': 2,
    'C2': 0.5,
    'C3': 1,
    'C4': np.inf
}

well_tf_ratio_status = pd.DataFrame({
    'Well': well_names,
    'Tum/Fib Ratio': [10, 0.1, 1.0, 1.0, 5.0, 0.2, 1.0, 0, 2.0, 0.5, 1.0, 0],
    'Status' : ['Lifted/Rotated', 'Stable/Adhered', 'Lifted/Rotated', 'Lifted/Rotated',
               'Lifted/Rotated', 'Lifted/Rotated', 'Lifted/Rotated', 'Lifted/Rotated',
               'Stable/Adhered', 'Lifted/Rotated', 'Stable/Adhered', 'Stable/Adhered']
})

#well_tf_ratio_status['Tum/Fib Ratio'] = pd.Categorical(well_tf_ratio_status['Tum/Fib Ratio'])  # Important: Makes it categorical
well_tf_ratio_status['Status'] = pd.Categorical(well_tf_ratio_status['Status'])
well_tf_ratio_status['Tum/Fib Ratio'] = [float(r) for r in  well_tf_ratio_status['Tum/Fib Ratio'].values]  # Important: Makes it numeric
print(well_tf_ratio_status)

In [ ]:
lifted_rotated = well_tf_ratio_status['Tum/Fib Ratio'][well_tf_ratio_status['Status'] == 'Lifted/Rotated']
stable_adhered = well_tf_ratio_status['Tum/Fib Ratio'][well_tf_ratio_status['Status'] == 'Stable/Adhered']

# Perform Kruskal-Wallis test
h, p = sp.stats.kruskal(lifted_rotated, stable_adhered)
p_value_text = str(p)
print(f"Kruskal-Wallis H statistic: {h}")
print(f"P-value: {p}")

In [ ]:
# figure parameters
title_fs = 22
annot_fs = 14
label_fs = 18
tick_fs = 14

fig, ax = plt.subplots(figsize=(6, 6))
sns.violinplot(
    x='Status', 
    y='Tum/Fib Ratio', 
    hue='Status', 
    palette = sns.color_palette('pastel'), 
    cut=0,
    data=well_tf_ratio_status,
    ax=ax
)
ax.set_title('Tumor/Fibroblast Ratio vs. Status', fontsize=title_fs)
ax.text(0.5, 0.95, f"p-value = {np.round(float(p_value_text), 2)}", 
         ha='center', va='top', transform=plt.gca().transAxes, fontsize=annot_fs) # Use gca().transAxes, adjust fontsize
ax.set_xlabel('Well Status', fontsize=label_fs)
ax.set_xticklabels(['Displaced', 'Static'], fontsize=tick_fs)
ax.set_ylabel('Tumor-to-CAF Ratio', fontsize=label_fs)
ax.set_ylim([-0.5, 12.0])
ax.set_yticks(np.arange(2, 12, 2))
ax.set_yticklabels(np.arange(2, 12, 2), fontsize=tick_fs)
fig.savefig(os.path.join(out_dir, "figures/tum_fib_ratio_test.png"), dpi=DPI, bbox_inches='tight')
plt.show()
plt.close()

## Figure 1F

In [ ]:
def calculate_circularity_eccentricity(binary_mask):
    """Calculates the circularity and eccentricity of the largest object in a binary mask.

    Args:
        binary_mask (numpy.ndarray): A 2D binary image (0 for background, 1 for objects).

    Returns:
        tuple: A tuple containing the circularity and eccentricity of the largest object 
               as floats, or (None, None) if no objects are found.
    """
    # Label connected regions (objects) in the binary mask
    labeled_mask = ski.measure.label(binary_mask)

    # If no objects are found, return None. Prevents an error in the next section if no objects are present.
    if labeled_mask.max() == 0:  # Check if any objects were found
        print("Warning: No objects detected in the binary mask.")
        return None, None


    # Calculate region properties for all labeled regions in the image.

    regions = ski.measure.regionprops(labeled_mask)


    # Find the largest object based on its area
    largest_object = max(regions, key=lambda region: region.area)

    # Calculate circularity
    circularity = 4 * np.pi * largest_object.area / (largest_object.perimeter ** 2)

    # Calculate eccentricity
    eccentricity = largest_object.eccentricity

    return circularity, eccentricity

def sort_filenames(filenames):
    """Sorts a list of filenames by initial digits and then by time point.

    Args:
        filenames (list): A list of filenames.

    Returns:
        list: The sorted list of filenames.
    """

    def extract_key(filename):
        """Extracts sorting keys from filename."""
        match = re.match(r"(\d+)_x\d+_(\d+)h", filename)  # Extract initial digits and time
        if match:
            initial_digits = int(match.group(1))
            time_point = int(match.group(2))
            return initial_digits, time_point
        else:
            return (float('inf'), float('inf'))  # Place non-matching filenames at the end

    sorted_filenames = sorted(filenames, key=extract_key) # Pass the key function to the sorted method.
    return sorted_filenames

In [ ]:
# Analyzing Spheroids from Rotated and Stable Wells
# Stable Spheroids: 18, 19, 20
stable_spheroids = ['18', '19', '20']

# Rotated Spheroids: 30, 31, 32
rotated_spheroids = ['30', '31', '32']

# all spheroids
all_spheroids = stable_spheroids + rotated_spheroids
all_spheroids

In [ ]:
timepoints = ['0h', '24h', '48h', '72h', '168h']

In [ ]:
mask_fnames = [f for f in os.listdir(data_dir) if "mask.tiff" in f]
mask_sorted_fnames = sort_filenames(mask_fnames)

In [ ]:
data = []
for i, spheroid in enumerate(all_spheroids):
    # get the masks for the spheroid
    spheroid_mask_fnames = [fn for fn in mask_sorted_fnames if spheroid in fn]
    # load the masks for the spheroid
    for t, timepoint in enumerate(timepoints):
        maskf = spheroid_mask_fnames[t]
        m = tifffile.imread(os.path.join(data_dir, maskf))
        circ, ecc = calculate_circularity_eccentricity(m)
        row_data = {
            'Spheroid': spheroid,
            'Well Status': "Well A3 (Displaced)" if spheroid in rotated_spheroids else "Well C3 (Static)",
            'Time Point': timepoint,
            'Eccentricity': ecc,
            'Circularity': circ
        }
        data.append(row_data)
df = pd.DataFrame(data)
# Convert 'Time Point' to categorical to maintain order if necessary.
df['Time Point'] = pd.Categorical(df['Time Point'], categories=["0h", "24h", "48h", "72h", "168h"], ordered=True)
df

In [ ]:
time_points = df['Time Point'].unique()
p_values = {}
for time in time_points:
    group1 = df[df['Time Point'] == time][df['Well Status'] == 'Well C3 (Static)']['Eccentricity']
    group2 = df[df['Time Point'] == time][df['Well Status'] == 'Well A3 (Displaced)']['Eccentricity']
    statistic, p_value = sp.stats.mannwhitneyu(group1, group2) # perform Mann-Whitney U test.
    p_values[time] = p_value

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

# generate the paired boxplot
sns.violinplot(
    x='Time Point', 
    y='Circularity', 
    hue='Well Status', 
    data=df,
    dodge=True, # Set dodge to False to make each bar overlayed rather than split for each well status.
    palette=[sns.color_palette('pastel')[1], sns.color_palette('pastel')[0]],#{'Stable/Adhered': 'skyblue', 'Rotated/Lifted': 'coral'}, 
    linewidth = 2, # set linewidth to make edge of box plot easier to visualize.
    width = 0.4,
    ax = ax
)

# Add stripplot to show raw data points
sns.swarmplot(
    x='Time Point',
    y='Circularity',
    hue='Well Status',
    data=df,
    dodge=True,  # Align with violinplot groups
    color='black',  # Set color of data points
    marker='o',
    size=8,       # Adjust marker size
    ax=ax
)

#get patchcollection for the sawrmplot
#c0 = ax.get_children()[1]
#x,y = np.array(c0.get_offsets()).T
#Add .2 to x values
#xnew=x+.2
#offsets = list(zip(xnew,y))
#set newoffsets
#c0.set_offsets(offsets)

# Add significance bars - Adjust y_start and increment
max_y = df['Circularity'].max()
y_start = max_y *1.5 #+ 0.02  # Start slightly above the highest data point. Remove the scaling by 1.05.
for i, time in enumerate(time_points):
    p = p_values[time]
    h = 0.01  # Fixed height of the bar. Reduced from 0.02 so it is easier to visualize within the ylimits of the axes.
    ax.plot([i - 0.2, i - 0.2, i + 0.2, i + 0.2], [y_start, y_start + h, y_start + h, y_start], lw=1.5, c='k') # Add h
    ax.text(
        i, 
        y_start + h, 
        "*" if p < 0.05 else "ns", 
        ha='center', 
        va='bottom', 
        fontsize=14, 
        fontweight='bold'
    ) #add annotation, add h
    #y_start += h + 0.01 # Increment based on the height of the bar + some extra spacing

ax.set_ylim(top=ax.get_ylim()[1]*1.05) # Expand the ylim slightly to fit annotations. # Multiply by a factor slightly >1.
ax.set_xlabel('Time Point', fontsize=18)
ax.set_xticklabels(timepoints, fontsize=16)
ax.set_ylabel('Invasiveness (Circularity)', fontsize=18)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=16)
ax.set_title('Invasiveness (Circularity) Over Time by Well Status', fontsize=22)

# Remove the "hue" legend for the swarmplot
# Get handles and labels from the violinplot legend
handles, labels = ax.get_legend_handles_labels()
# Use the first half of the handles and labels (corresponding to the violinplot)
# since the second half will correspond to the swarmplot for each hue level, which we want to exclude.
ax.legend(
    handles[:2], 
    labels[:2], 
    title="Well Status", 
    fontsize=14, 
    title_fontsize=16,
    loc='lower left'
)

plt.setp(ax.get_legend().get_texts(), fontsize='14') # for legend text
plt.setp(ax.get_legend().get_title(), fontsize='16') # for legend title

fig.savefig(os.path.join(out_dir, "invasiveness_circularity_comparison.png"), dpi = 300, bbox_inches='tight')
plt.show()
plt.close()